## Monarch + TorchTitan on SLURM
This example notebook demonstrates how you can easily run and iterate on a distributed training job with Monarch and TorchTitan.

#### Prerequisites
Please make sure your environment is setup for this notebook:
1. Install Monarch nightly: https://github.com/meta-pytorch/monarch/blob/main/scripts/install_nightly.py
2. Install Titan nightly: https://github.com/pytorch/torchtitan?tab=readme-ov-file#nightly-builds
3. Ensure you have a valid Titan model config in the script directory (i.e: https://github.com/pytorch/torchtitan/blob/main/torchtitan/models/llama3/train_configs/debug_model.toml)

### 1. Reserve your SLURM job
If necessary, update paramaters for your cluster:
- host_type: TorchX named resource for your cluster (default: "gpu.xlarge")
- host_memory: Memory per machine in MB (default: 2062607)

For more information on TorchX resources: https://docs.pytorch.org/torchx/main/specs.html#resource

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

from slurm.utils import get_appdef, get_server_info, create_proc_mesh


num_nodes = 2 # assign for your system
appdef = await get_appdef(
    num_nodes,
    # host_type = ...
)
server_info = await get_server_info(
    appdef,
    # host_memory = ...
)

<string>:8: FutureWarning: Setting `workspace='/mnt/home/amirafzali/monarch/examples'` is deprecated. Use `workspace=monarch.tools.config.workspace.Workspace(dirs=['/mnt/home/amirafzali/monarch/examples'])` instead.
torchx.schedulers.slurm_scheduler 2025-09-23 04:50:42 INFO unable to get job info for `monarch-amirafzali` with `squeue` (squeue: error: Invalid job id: monarch-amirafzali
), trying `sacct`
torchx.schedulers.slurm_scheduler 2025-09-23 04:50:42 INFO unable to get job info for `monarch-amirafzali` with `sacct` (sacct: fatal: Bad job/step specified: monarch-amirafzali
)
monarch.tools.commands 2025-09-23 04:50:42 INFO no existing RUNNING server `slurm:///monarch-amirafzali` creating new one...


torchx.runner.api 2025-09-23 04:50:42 INFO Tracker configurations: {}
torchx.runner.api 2025-09-23 04:50:42 INFO Checking for changes in workspace `/mnt/home/amirafzali/.monarch/out/tmptsrcdyex/workspace`...
torchx.runner.api 2025-09-23 04:50:42 INFO To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.
torchx.runner.api 2025-09-23 04:50:42 INFO Reusing original image `monarch_default_workspace:latest` for role[0]=mesh0. Either a patch was built or no changes to workspace was detected.
monarch.tools.commands 2025-09-23 04:50:42 INFO created new `slurm:///295` waiting for it to be ready...


New job `slurm:///295` is ready to serve.urrent: PENDING); will check again in 5.0 seconds. Total wait time: 0:00:25.251284


### 2. Define your Titan and cluster parameters

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

from torchtitan.train import Trainer
from torchtitan.config import ConfigManager, JobConfig
from monarch.actor import Actor, current_rank, endpoint
from torchtitan.tools.logging import init_logger, logger
import torch
from dataclasses import dataclass
import os
from monarch.tools import commands
from monarch.utils import setup_env_for_distributed


@dataclass
class RunParams:
    """
        Parameters for your cluster and training job, adjust as needed
    """
    training_steps: int = 50
    model_config = "debug_model.toml"
    dataset = "c4"
    num_nodes = num_nodes
    gpus_per_node = 4


class TrainerActor(Actor):
    """
        A simple wrapper class with executes a TorchTitan trainer in a Monarch actor
    """
    def __init__(self, job_config: JobConfig) -> None:
        self.job_config = job_config
        rank = current_rank().rank
        self.uid = f"[trainer_{rank}]"

    @endpoint
    async def start_training(self) -> None:
        init_logger()
        trainer: Trainer | None = None

        try:
            trainer = Trainer(self.job_config)
            logger.info(f"{self.uid} initialized successfully and starting training")
            trainer.train()
        except Exception:
            if trainer:
                trainer.close()
            raise
        else:
            trainer.close()
        finally:
            torch.distributed.destroy_process_group()
            logger.info(f"{self.uid} trainer cleaned up")

def make_job_config() -> JobConfig:
    """
        Create a job config which is digested by TorchTitan, sourced from RunParams
    """
    data_parallel_shard_degree = RunParams.num_nodes * RunParams.gpus_per_node
    output_path = "./outputs"

    script_dir = globals()['_dh'][0]
    default_args = [
        "--job.config_file",
        os.path.join(script_dir, RunParams.model_config),
        "--model.tokenizer_path",
        os.path.join(script_dir, "tokenizer"),
        "--comm.trace_buf_size",
        "0",
        "--metrics.log_freq",
        "1",
        "--parallelism.data_parallel_shard_degree",
        str(data_parallel_shard_degree),
        "--activation_checkpoint.mode",
        "full",
        "--comm.train_timeout_seconds",
        "60",
        "--training.steps",
        str(RunParams.training_steps),
        "--training.dataset",
        RunParams.dataset,
        "--job.dump_folder",
        output_path,
        "--metrics.enable_tensorboard",
    ]

    config_manager = ConfigManager()
    job_config = config_manager.parse_args(default_args)

    return job_config

### 3. Execute your training job
You can make adjustments and run this on the existing SLURM allocations as many times as you would like!

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

async def main():
    job_config = make_job_config()
    proc_mesh = None

    try:
        # 1. Create a proc mesh on your SLURM allocation
        proc_mesh = await create_proc_mesh(RunParams.num_nodes, appdef, server_info)
        # 2. Define remote logging behavior
        await proc_mesh.logging_option(
            stream_to_client=True
            # aggregate_window_sec=None
        )
        # 3. Prepare trainer for torch distributed
        await setup_env_for_distributed(proc_mesh)
        trainer = await proc_mesh.spawn("trainer_actor", TrainerActor, job_config)
        # 4. Execute the taining job
        await trainer.start_training.call()
    except Exception as e:
        logger.info(f"Trainer failed: {e}")
    finally:
        if proc_mesh:
            await proc_mesh.stop()


if __name__ == "__main__":
    await main()

root 2025-09-23 04:51:17 WARNING tokenizer_path is deprecated, use model.hf_assets_path instead. Setting hf_assets_path to tokenizer_path temporarily.
slurm.utils 2025-09-23 04:51:17 INFO 
===== Server Info =====
{
  "name": "295",
  "server_handle": "slurm:///295",
  "state": "RUNNING",
  "meshes": {
    "mesh0": {
      "host_type": "__UNSET__",
      "hosts": 2,
      "gpus": -1,
      "hostnames": [
        "slurm-h100-206-061",
        "slurm-h100-206-089"
      ]
    }
  }
}
monarch._src.actor.allocator 2025-09-23 04:51:17 INFO no match label `procmesh.monarch.meta.com/name` specified in alloc constraints
monarch._src.actor.allocator 2025-09-23 04:51:17 INFO found a single proc mesh `mesh0` in slurm:///295, will allocate on it
monarch.tools.network 2025-09-23 04:51:17 INFO no AF_INET6 address that can bind TCP sockets for `slurm-h100-206-061:26600` (error: [Errno -5] No address associated with hostname)
monarch.tools.network 2025-09-23 04:51:17 INFO resolved AF_INET address `addr

monarch._src.actor.allocator 2025-09-23 04:51:17 INFO no match label `procmesh.monarch.meta.com/name` specified in alloc constraints
monarch._src.actor.allocator 2025-09-23 04:51:17 INFO found a single proc mesh `mesh0` in slurm:///295, will allocate on it
monarch.tools.network 2025-09-23 04:51:17 INFO no AF_INET6 address that can bind TCP sockets for `slurm-h100-206-061:26600` (error: [Errno -5] No address associated with hostname)
monarch.tools.network 2025-09-23 04:51:17 INFO resolved AF_INET address `address1:26600` for `slurm-h100-206-061:26600`
monarch.tools.network 2025-09-23 04:51:17 INFO no AF_INET6 address that can bind TCP sockets for `slurm-h100-206-089:26600` (error: [Errno -5] No address associated with hostname)
monarch.tools.network 2025-09-23 04:51:17 INFO resolved AF_INET address `address2:26600` for `slurm-h100-206-089:26600`
monarch._src.actor.allocator 2025-09-23 04:51:17 INFO initializing alloc on remote allocator addresses: ['tcp!address1:26600', 'tcp!address2:26

>>> Aggregated Logs (2025-09-23 04:51:21) >>>
[1 similar log lines] [titan] 2025-09-23 04:51:26,853 - root - INFO - Starting job: Llama 3 debug training
<<< Aggregated Logs (2025-09-23 04:51:26) <<<

>>> Aggregated Logs (2025-09-23 04:51:26) >>>
[7 similar log lines] [titan] 2025-09-23 04:51:26,853 - root - INFO - Starting job: Llama 3 debug training
[8 similar log lines] [titan] 2025-09-23 04:51:27,460 - root - INFO - Building 1-D device mesh with ['dp_shard'], [8]
[8 similar log lines] [titan] 2025-09-23 04:51:27,463 - root - INFO - [GC] Initial GC collection 0.00 seconds
[8 similar log lines] [titan] 2025-09-23 04:51:29,199 - root - INFO - Loading tokenizer from tokenizer.json
[8 similar log lines] [titan] 2025-09-23 04:51:29,201 - root - INFO - Preparing c4 dataset from allenai/c4
<<< Aggregated Logs (2025-09-23 04:51:29) <<<

>>> Aggregated Logs (2025-09-23 04:51:29) >>>
[1 similar log lines] [titan] 2025-09-23 04:51:33,920 - root - INFO - Building llama3 debugmodel with Transform

[-]E0923 04:51:44.398610 2147845 hyperactor_mesh/src/alloc/remoteprocess.rs:1088] failed to cleanup disconnected host: got channel closed event for host address1 which has no known state


### 4. Destory the SLURM job
Once you're done experimenting, free up the allocation

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

commands.kill(f"slurm:///{server_info.name}")